# Projeto 2 - Ciência dos Dados

Nome: Thomas Bekhor

Nome: Luca Machado

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import display
pd.options.display.max_rows = 13

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [32]:
#Produto escolhido:
produto = "Uber"

#Quantidade mínima de mensagens capturadas:
n = 590
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [33]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
msgs = list(dict.fromkeys(msgs))
shuffle(msgs)


Salvando os dados em uma planilha Excel:

In [34]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)
    
    #fecha o arquivo
    writer.save()
    

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

# 1=RELEVANTE
# 2=IRRELEVANTE

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [4]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [5]:
#lendo cada planilha separadamente do excel
twitter_treinamento = pd.read_excel(io="./Uber.xlsx",sheet_name="Treinamento")
twitter_teste = pd.read_excel(io="./Uber.xlsx",sheet_name="Teste")

#aplicando a função cleanup no "Teste" e tirando as maiúsculas
twitter_teste["Teste s/ pont."] = twitter_teste["Teste"].apply(cleanup)
twitter_teste=twitter_teste.drop(columns="Teste")
twitter_teste=twitter_teste[["Teste s/ pont.", "Relevância"]]
twitter_teste["Teste s/ pont."]=twitter_teste["Teste s/ pont."].str.lower()

#aplicando a função cleanup no "Treinamento"  e tirando as maiúsculas
twitter_treinamento["Treinamento s/ pont."] = twitter_treinamento["Treinamento"].apply(cleanup)
twitter_treinamento=twitter_treinamento.drop(columns="Treinamento")
twitter_treinamento=twitter_treinamento[["Treinamento s/ pont.", "Relevância"]]
twitter_treinamento["Treinamento s/ pont."]=twitter_treinamento["Treinamento s/ pont."].str.lower()



In [6]:
#separando as palaras numa lista só
series_treinamento=[]
for i in twitter_treinamento["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento.append(n)
series_treinamento

#separando as palavras para os relevantes
series_treinamento_rel=[]
for i in twitter_treinamento[twitter_treinamento["Relevância"]==1]["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento_rel.append(n)

#separando as palavras para os não relevantes
series_treinamento_nrel=[]
for i in twitter_treinamento[twitter_treinamento["Relevância"]==0]["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento_nrel.append(n)


In [7]:
#frequencia relativa nao relevantes
frame_rel=pd.Series(series_treinamento_rel)
frame_rel_relativo=frame_rel.value_counts(True)

#frequencia relativa relevantes
frame_nrel=pd.Series(series_treinamento_nrel)
frame_nrel_relativo=frame_nrel.value_counts(True)

#frequencia relativa total
frame_total=pd.Series(series_treinamento)
frame_total_relativo=frame_total.value_counts(True)

frame_nrel_relativo

uber          0.053293
o             0.030539
e             0.028743
eu            0.023952
de            0.022754
que           0.022754
                ...   
casaa         0.000299
aplicativo    0.000299
2min          0.000299
branco        0.000299
terminou      0.000299
z             0.000299
Length: 1266, dtype: float64

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

$P(relevante|frase) > P(irrelevante|frase)$

$P(relevante|frase) = \frac{P(palavra1|relevante).P(palavra2|relevante).P(palavra3|relevante).P(relevante)}{P(frase)}$

$P(irrelevante|frase) = \frac{P(palavra1|irrelevante).P(palavra2|irrelevante).P(palavra3|irrelevante).P(irrelevante)}{P(frase)}$

# logo
$P(relevante|frase)> P(irrelevante|frase)$ =
$\frac{P(palavra1|relevante).P(palavra2|relevante).P(palavra3|relevante).P(relevante)}{P(frase)}$ > $\frac{P(palavra1|irrelevante).P(palavra2|irrelevante).P(palavra3|irrelevante).P(irrelevante)}{P(frase)}$

In [9]:

lista_prob_rel=[]
lista_prob_nrel=[]

#laplace=

for i in range(0,200):
    prf=0.39
    pif=0.61
    for x in twitter_teste["Teste s/ pont."][i].split():
        if x in series_treinamento_rel:
            prf=prf*frame_rel_relativo[x]
        if x in series_treinamento_nrel:    
            pif=pif*frame_nrel_relativo[x]         
        else:
            prf=prf*2
            pif=pif*2
            
    lista_prob_rel.append(prf)
    lista_prob_nrel.append(pif)
    
for x in range(0,200):
    if lista_prob_rel[x]>lista_prob_nrel[x]:
        twitter_teste.loc[twitter_teste["Teste s/ pont."][x],"Verificação"]=1#problema2
    else:
        twitter_teste.loc[twitter_teste["Teste s/ pont."][x],"Verificação"]=0
twitter_teste

,Teste s/ pont.,Relevância,Verificação
0,tava tão doida p ir embora daquele lugar q ent...,0.0,NaN
1,inclusive quando vi o da campus party quase fa...,0.0,NaN
2,status cantando sertanejo com o uber,1.0,NaN
3,uber bom é uber que bate o carro no cara de trás,1.0,NaN
4,e eu q pedi um uber aí tava nome de um homem l...,1.0,NaN
5,trêm//metrô 4 30\nuber 16 90\n\neu quase o ...,1.0,NaN
...,...,...,...
decidi pegar uber,NaN,NaN,0.0
uber sem ar condicionado é sacanagi,NaN,NaN,0.0
tô muito mau acostumada c uber esse fim de semana c hj gastei 200 e pouco perai eu tô ficando louca,NaN,NaN,1.0


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**